In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.MIP.mip_inputs import *
from src.optimization.MIP.mip_model import *
from src.optimization.MIP.mip_outputs import *

2021-04-13 21:57:25.728 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-04-13 21:57:25.730 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
start = time.time()

In [3]:
x = np.array([1,1,1])

In [4]:
x

array([1, 1, 1])

In [5]:
np.append(x,np.zeros((14-x.shape[0],)))#.shape

array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Model Inputs 

In [9]:
UI_params = {
    'initial_procurement_budget':1300000,
    'budget_rate':0.03,
    'initial_operations_budget':1000000,
    'maintenance_budget_rate':0.03,
    'planning_interval':[2022,2037],
    'emissions_baseline': 2031705,#metric tons
    'emissions_target_pct_of_base':0.10,
    'min_miles_replacement_threshold':150000,#miles
    'min_vehicle_age_replacement_threshold':60,#years
    'max_vehicles_per_station':1000,
    'objective_weights':{'cost':0.70,'emissions':0.30},
}

In [10]:
data = pd.read_excel('../data/17MAR_data_template.xlsx').head(25)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [11]:
inputs = MIP_Inputs(data,UI_params)

In [21]:
inputs.replacement_schedules.shape

(25, 697, 16)

In [26]:
inputs.years

[2022,
 2023,
 2024,
 2025,
 2026,
 2027,
 2028,
 2029,
 2030,
 2031,
 2032,
 2033,
 2034,
 2035,
 2036,
 2037]

In [25]:
inputs.replacement_schedules[1,[1,10,34],:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.]])

In [12]:
(inputs.budget_acquisition + (inputs.budget_acquisition*np.indices((16,))*.03)).flatten()

array([1300000., 1379170., 1460680., 1544530., 1630720., 1719250., 1810120., 1903330., 1998880., 2096770., 2197000., 2299570., 2404480., 2511730., 2621320., 2733250.])

In [9]:
1300000*.03

39000.0

In [13]:
np.indices((16,))*.03

array([[0.  , 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3 , 0.33, 0.36, 0.39, 0.42, 0.45]])

In [6]:
outputs = MIP_Outputs(data,UI_params)
print("--- %s seconds ---" % (time.time() - start))

TypeError: __init__() missing 7 required positional arguments: 'm', 'x', 'vehicles', 'penalty_budget', 'penalty_emissions', 'validSchedulesPerVehicle', and 'validSchedules'

In [ ]:
optimal_obj,optimal_solution = outputs.get_optimal_solution()

In [ ]:
alt_solutions = outputs.get_alternative_solutions()
selected_alternative_solutions = outputs.select_alternative_solutions(alt_solutions,optimal_obj)

In [ ]:
options = selected_alternative_solutions

In [ ]:
option1 = pd.DataFrame(options[0,'schedules'],columns=['vehicle_idx','solution_idx']+inputs.years)

In [ ]:
data = data.merge(option1,on='vehicle_idx',how='left')

In [ ]:
data.groupby(['county'])[inputs.years].sum().replace({0:''})

In [ ]:
data.groupby(['dept_name'])[inputs.years].sum().replace({0:''}).head()

In [ ]:
data.groupby(['dept_name','vehicledescription'])[inputs.years].sum().replace({0:''}).head(20)

In [ ]:
data.groupby(['equipmentid'])[inputs.years].sum().replace({0:''}).sample(20)

In [ ]:
print('   acquisition_costs')
pd.DataFrame(pd.DataFrame(options[0,'acquisition_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

In [ ]:
print('   consumables_costs')
pd.DataFrame(pd.DataFrame(options[0,'consumables_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

In [ ]:
print('   mx_costs')
pd.DataFrame(pd.DataFrame(options[0,'mx_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

In [ ]:
print('   emissions_amts')
pd.DataFrame(pd.DataFrame(options[0,'emissions_amts'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

In [ ]:
print('   EV/Hybrid Inventory')
ev_inv = pd.DataFrame(pd.DataFrame(options[0,'conversions'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum())
ev_inv.columns = ['EV']
ev_inv.EV = ev_inv.EV
ev_inv['ICE'] = inputs.num_vehicles-ev_inv.EV
# ev_inv.melt(value_vars=['EV','ICE'],ignore_index=False)

In [ ]:
import plotly.graph_objects as go



fig = go.Figure()
fig.add_trace(go.Bar(x=inputs.years,
                y=ev_inv.EV,
                name='EV/Hybrid',
                marker_color='rgb(15,157,88)'
                ))
fig.add_trace(go.Bar(x=inputs.years,
                y=ev_inv.ICE,
                name='ICE',
                marker_color='rgb(0,0,0)'
                ))


In [ ]:
ev_inv/inputs.num_vehicles

In [ ]:
inputs.num_vehicles